In [244]:
import pandas as pd
import numpy as np
import plotly.express as px
import pandas_market_calendars as mcal

## Payoff Function

In [218]:
def payoff(paths, K, Barrier, ratio):
    Nsim = paths.shape[0]
    final_prices = paths[:, :, -1] #S_T
    above_strike = np.all(final_prices >= K, axis=1)
    barrier_event = np.any(paths.min(axis=2) <= Barrier, axis=1)

    # No barrier event or (barrier event and above strike)
    cash_payout = np.where(np.logical_or(np.logical_not(barrier_event), np.logical_and(barrier_event,  above_strike)), 1105, 0)

    # barrier event and below strike
    percent_change = final_prices / K # Based on final strike price
    min_loc = np.argmin(percent_change, axis=1) # Finding the worst performing underlying
    j = np.indices(min_loc.shape)
    payouts = ratio[min_loc] * final_prices[j, min_loc] + 105 # certificate to stock ratio * final price of the stock + coupon payment
    stock_payout = np.where(np.logical_and(barrier_event,  np.logical_not(above_strike)), payouts, 0)

    return np.sum(np.concatenate((cash_payout.reshape(1, Nsim), stock_payout.reshape(1, Nsim))), axis=0)

Expected Payoff: $ \hat{f}(S, t) = \frac{1}{n} \sum^n_{i=1} e^{-r(T-t)} \chi(S^{(i)})$

In [219]:
def expected_payoff(payoffs, r, T, t):
    res = []
    discount_factor = np.exp(-r*(T-t))
    mean_payoff = np.mean(payoffs)
    
    return discount_factor * mean_payoff

## Plotting

In [220]:
def plot_simulations(price_hist, Nsim, sim_paths, stock=0):
    """plots simulations for chosen stock

    Args:
        price_hist (df): df of historical prices
        sim_paths (list): simulated price paths
        stock (int, optional): index of stock. Defaults to 0.
    """
    # df of historical prices
    price_hist = price_hist.to_numpy()[:, [0]].reshape(-1)
    price_hist = np.vstack([price_hist]*Nsim)
    price_hist = pd.DataFrame(price_hist).transpose()

    sim_paths = pd.DataFrame(sim_paths[:, stock, :]).transpose()
    sim_paths = pd.concat((price_hist, sim_paths))
    sim_paths = sim_paths.reset_index(drop=True)
    fig = px.line(sim_paths)
    return fig

# GBM Simlulations


## Standard MC

In [221]:
def multi_asset_GBM(S0, v, Sigma, delta_t, m, p):
    """GBM simulation of multiple asset paths

    Args:
        S0 (vector of start prices): vector of starting prices
        v (array): mu/dt
        Sigma (matrix): covariance matrix
        delta_t (float): delta between each time step
        m (int): number of days simulated
        p (int): number of assets
    """

    S = np.zeros(shape=(m, p))
    S[0] = S0

    z = np.random.multivariate_normal(mean=v*delta_t, cov=Sigma*delta_t, size=m)
    for step in range(1, m):
        S[step] = np.exp(np.log(S[step-1]) + z[step-1])
    return np.transpose(S)

## Antithetic Variate

In [222]:
def multi_asset_GBM_av(S0, v, Sigma, delta_t, m, p):
    """GBM simulation of multiple asset paths

    Args:
        S0 (vector of start prices): vector of starting prices
        v (array): mu/dt
        Sigma (matrix): covariance matrix
        delta_t (float): delta between each time step
        m (int): number of days simulated
        p (int): number of assets
    """

    S = np.zeros(shape=(m, p))
    Stilde = np.zeros(shape=(m, p))
    S[0] = S0
    Stilde[0] = S0

    z = np.random.multivariate_normal(mean=v*delta_t, cov=Sigma*delta_t, size=m)
    for step in range(1, m):
        S[step] = np.exp(np.log(S[step-1]) + z[step-1])
        Stilde[step] = np.exp(np.log(Stilde[step-1]) - z[step-1])
    return np.transpose(S), np.transpose(Stilde)

## Helper Functions

In [224]:
# function to get v and Sigma from within a 1 year window
def get_simulation_params(asset_hist, current_date, dt):
    start_date = current_date - pd.Timedelta(365, "day")
    prices = asset_hist.loc[start_date:current_date]
    log_prices = prices.apply(lambda x: np.log(x))
    log_returns = log_prices.apply(lambda x: np.diff(x))
    v = (log_returns.mean()/dt).values
    Sigma = (log_returns.cov()/dt).to_numpy()
    return v, Sigma

def n_path_sim(asset_hist, Nsim, current_date, T, dt, S0, remaining_steps, sim_func, var_reduction=None):
    p = len(S0)
    print(f"Simulating {Nsim} paths for {p} assets, starting on {current_date}")

    if var_reduction == None:
        paths = np.zeros(shape=(Nsim, p, remaining_steps))
        for i in range(Nsim):
            v, Sigma = get_simulation_params(asset_hist, current_date, dt)
            paths[i] = sim_func(S0, v, Sigma, dt, remaining_steps, p)

    elif var_reduction == "av":
        paths = np.zeros(shape=(2 * Nsim, p, remaining_steps))
        sims = sim_func(S0, v, Sigma, dt, remaining_steps, p)
        for i in range(Nsim):
            v, Sigma = get_simulation_params(asset_hist, current_date, dt)
            paths[i], paths[Nsim + i] = sim_func(S0, v, Sigma, dt, remaining_steps, p)
 
    return paths
        
        

# Run Simulations

In [225]:
df = pd.read_csv("assets.csv", index_col='Date')
df.index = df.index.astype("datetime64")

In [252]:
# Create a calendar
nyse = mcal.get_calendar('NYSE')

# set vars for product lifetime
initial_fixing = pd.to_datetime("2021-05-25")
maturity = pd.to_datetime("2022-11-29")
lifetime = nyse.schedule(start_date=initial_fixing, end_date=maturity).index
S0 = df.loc[initial_fixing].values

# gathered from product description
K = np.array([103.87, 413.05, 58.26])
Barrier = np.array([62.322, 247.830, 34.956])
ratio = np.array([9.6274, 2.4210, 17.1644])

# historical prices as percentage change from initial fixing
asset_hist = df.divide(S0/100)

m = len(lifetime)  # no of days product is active for 
T = m/252  # period in terms of no. of financial years
dt = 1/252  # daily increment
t = 0  # initial start time is 0

In [253]:
# ROLLING WINDOW OVER ENTIRE LIFETIME
# uses newest asset prices on current day to get expected payoff for that day
# make sure to run previous cell before running this one
expected_payoffs = np.zeros(len(lifetime))
for day in lifetime:
    paths = n_path_sim(asset_hist=asset_hist,
                    Nsim=10,
                    current_date=day,
                    T=T,
                    dt=dt,
                    S0=S0,
                    remaining_steps=m-t,
                    sim_func=multi_asset_GBM)

    expec_payoff = expected_payoff(payoff(paths, K, Barrier, ratio), 0.04, 1, 1/252)
    expected_payoffs[t] = expec_payoff
    # track step increment
    t += 1

Simulating 10 paths for 3 assets, starting on 2021-05-25 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-05-26 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-05-27 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-05-28 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-01 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-02 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-03 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-04 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-07 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-08 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-09 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-10 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-11 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-14 00:00:00
Simulating 10 paths for 3 assets, starting on 2021-06-15 00:00:00
Simulating

In [254]:
print(len(lifetime))
print(len(expected_payoffs))

383
383
